<a href="https://colab.research.google.com/github/ajcommercial/AI_color_grade_lut/blob/master/BatchPix2LUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Batch convert image pairs to LUTs

This notebook converts a batch of image pairs (prepared in the same fashion as for pix2pix) into LUTs, that are grading the input images towards the corresponding target images.

The images need to be uploaded in a zipped folder. After the program is finished with all files, it creates a folder "*name of your zipfolder*_LUT.zip" that you can download



In [24]:

import glob
import os
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
from IPython.display import clear_output 
import time
import tensorflow as tf
import numpy as np
from PIL import Image
from sklearn import neighbors
import zipfile

In [30]:

def img_load(fname):
    img = Image.open(fname)
    img = np.array(img)

    return img


def load(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image)

    input_image = tf.cast(image, tf.float32)

    return input_image


def resize(input_image, size):
    output_image = tf.image.resize(input_image, [size, size],
                                   method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return output_image


def normalize(input_image):
    output_image = (input_image / 127.5) - 1

    return output_image


def img_to_lut(input_image, target_image):
    Target = target_image * 0.5 + 0.5
    Input = input_image * 0.5 + 0.5

    safe = Target
    x = 0
    tR = []
    tG = []
    tB = []
    shape = Target.shape

    # differences on each pixel; here all pixels of the ixj array get sorted along
    # one dimension (i*j long) for both the input and the target image in the same
    # way. Afterwards for every color channel (tR, tG, TB) a list is created that
    # contains the difference between target value and input value.
    for i in range(shape[0]):
        for j in range(shape[1]):
            tR.append(Target[i, j, 0] - Input[i, j, 0])
            tG.append(Target[i, j, 1] - Input[i, j, 1])
            tB.append(Target[i, j, 2] - Input[i, j, 2])

            # here a list is created that sorts the input RGB Values along the pixel numbers
    # in the same order as above. So it starts with the first line on the left, once
    # the line is finished, counting continues on the next line left
    icolor = []
    for i in range(shape[0]):
        for j in range(shape[1]):
            icolor.append(Input[i, j, :])
    icolor = np.asarray(icolor)

    #interpolation = neighbors.RadiusNeighborsRegressor(radius = 0.06, weights='distance')

    LUT_size = 8
    a = LUT_size - 1
    # here the input LUT is created out of equally spaced RGB values in R^3
    LUT = []
    # LUT.append(['TITLE', '"test"', ''])
    # LUT.append(['LUT_3D_SIZE', str(LUT_size), ''])
    # LUT.append(['', '', ''])
    # LUT.append(['', '', ''])
    for k in range(LUT_size):
        for j in range(LUT_size):
            for i in range(LUT_size):
                LUT.append([i / a, j / a, k / a])

    # since we have all lists in the same order (icolor and the color differences tR,
    # tG, tB) we now can fit a function between icolor and the coresponding color
    # difference at each RGB coordinate. Afterwrds we predict the difference of the
    # LUT that is on a grit at this position

    interpolation = neighbors.KNeighborsRegressor(10, weights='distance')
    LUT = np.asarray(LUT)
    tRp = interpolation.fit(icolor, tR).predict(LUT)
    tGp = interpolation.fit(icolor, tG).predict(LUT)
    tBp = interpolation.fit(icolor, tB).predict(LUT)
    # print(tGp)
    for i in range(LUT_size ** 3):
        LUT[i, 0] = LUT[i, 0] + tRp[i]
        # print(tBp[i])
        LUT[i, 1] = LUT[i, 1] + tGp[i]
        LUT[i, 2] = LUT[i, 2] + tBp[i]

    LUT = LUT.clip(min=0)
    return LUT


def processing(file_path):
    img_size = 32



    image = load(file_path)  # preprocess the image
    image = normalize(image)
    w = image.shape[1]//2
    target_image = resize(image[:, :w, :],img_size)
    input_image = resize(image[:, w:, :],img_size)


    print('start to process LUT')
    LUT = img_to_lut(input_image, target_image)
    base= os.path.basename(file_path)
    filename = os.path.splitext(base)[0] 
    output = 'content/LUT/' + filename + '.cube'
    print(output)
    file = open(output, "w")
    file.write('TITLE "test"\n')
    file.write('LUT_3D_SIZE 8')
    file.write('\n')
    file.write('\n')
    for i in range(len(LUT)):
        bstr = str(int(LUT[i][0]*10000) / 10000) + ' ' + str(int(LUT[i][1]*10000) / 10000) + ' ' + str(int(LUT[i][2]*10000) / 10000) + '\n'
        file.write(bstr)
    file.close()
    # np.savetxt(output, LUT, fmt="%2.4f", delimiter=" ")
    return output


# check whether file is in allowed images or not
def encode(string):
    filename, file_extension = os.path.splitext(string)
    timestamp = str(time.time()).replace('.','_')
    return filename + '_' + timestamp + file_extension




In [ ]:
################################################################################

reverse = False
advanced = False

max_factor = 2
################################################################################

os.chdir("/")

for file in glob.glob("/content/*.zip"):
    print(file)
    path_to_zip_file = file

base= os.path.basename(path_to_zip_file)
filename = os.path.splitext(base)[0] 

PATH = os.path.join('/content/',filename+ '/')

print(PATH)
try:
    os.mkdir('/content/LUT')
except OSError:
    print ("Directory  already exists" )
else:
    print ("Successfully created the directory " )

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(PATH)

files = (glob.glob(os.path.join(PATH, '*jpg')))
num_files = len(files)
i = 0

for file in files:
  i = i +1
  clear_output()
  print("processing file " + str(i)+ " of " + str(num_files))
  output = processing(file)

def zipdir(path, ziph):
    # ziph is zipfile handle
   files = (glob.glob(os.path.join('/content/LUT', '*cube')))
   for file in files:
      ziph.write(file)


zipf = zipfile.ZipFile('/content/'+filename+'_LUTs.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('/content/LUT/', zipf)
zipf.close()


